**План проекта**
1. Скачать и распарсить корпус статей с горного журнала: БВР, ОГР, ПГР, обогащение, современные технологии
2. Подготовить словари  - термины и общая лексика
3. Чистка корпуса 
4. Токенизация 
5. Стоп-слова
6. Найти в корпусе слова, в которых Pymorphy выдает разные части речи для одного слова - **не сделано**
7. Найти совпадения в списке токенов из корпуса и в словаре терминов
8. Сделать частотный анализ tf-idf для определения тематики текста
9. Написать парсер вордовских файлов по выбору пользователя для автоматического выполнения п. 3-8






# 1. Скачивание корпуса
статьи с сайта https://mining-media.ru/


In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import datetime as dt
import re
import os

In [ ]:
# functions

# считываем код страницы
def parsing_pipe(url):
    req = requests.get(url)
    content = req.content
    soup = bs(content, 'html.parser')
    return soup

# собираем ссылки
def parse_link(soup, topic):
    articles = []
    listlinks = soup.find_all("a", href = True)
    for link in listlinks:
      if re.search(topic, link["href"]) != None:
        link = 'https://mining-media.ru/'+ link["href"]
        article_text = parse_article(link)
        #articles.append(link)
    return article_text #set(articles)

# идем по ссылке и собираем текст статьи
def parse_article(url):
    pagesoup = parsing_pipe(url)
    title = pagesoup.find("h1").get_text()
    text = " ".join([i.get_text() for i in pagesoup.find_all("p")])
    
    return title + '\n' + text + '\n\n'       

In [ ]:
# ОГР
url_start = "https://mining-media.ru/ru/article/ogr?start="
topic = 'ru/article/ogr/' 

i = 10
# идем по списку статей (по 10 на каждой странице) и сохраняем текст статей в файл
while i < 120 :
  url = url_start + str(i)
  pagecontent = parsing_pipe(url)
  article_text = parse_link(pagecontent, topic)
  i += 10
  with open('OP.txt', 'a', encoding = 'utf-8') as f:
      f.write(article_text)

In [ ]:
# БУРОВОЕ
url_start = "https://mining-media.ru/ru/article/burovoe?start="
topic = 'ru/article/burovoe/' 


i = 10
# идем по списку статей (по 10 на каждой странице) и сохраняем текст статей в файл
while i < 150 :
  url = url_start + str(i)
  pagecontent = parsing_pipe(url)
  article_text = parse_link(pagecontent, topic)
  i += 10
  with open('drilling.txt', 'a', encoding = 'utf-8') as f:
      f.write(article_text)

In [ ]:
# ПГР
url_start = "https://mining-media.ru/ru/article/podzemn?start="
topic = 'ru/article/podzemn/'

i = 10
# идем по списку статей (по 10 на каждой странице) и сохраняем текст статей в файл
while i < 150 :
  url = url_start + str(i)
  pagecontent = parsing_pipe(url)
  article_text = parse_link(pagecontent, topic)
  i += 10
  with open('UG.txt', 'a', encoding = 'utf-8') as f:
      f.write(article_text)

In [ ]:
# ОБОГАЩЕНИЕ
url_start = "https://mining-media.ru/ru/article/obogach?start="
topic = 'ru/article/obogach/'

i = 10
# идем по списку статей (по 10 на каждой странице) и сохраняем текст статей в файл
while i < 150 :
  url = url_start + str(i)
  pagecontent = parsing_pipe(url)
  article_text = parse_link(pagecontent, topic)
  i += 10
  with open('processing.txt', 'a', encoding = 'utf-8') as f:
      f.write(article_text)

In [ ]:
# НОВЫЕ ТЕХНОЛОГИИ
url_start = "https://mining-media.ru/ru/article/newtech?start="
topic = 'ru/article/newtech/'

i = 10
# идем по списку статей (по 10 на каждой странице) и сохраняем текст статей в файл
while i < 150 :
  url = url_start + str(i)
  pagecontent = parsing_pipe(url)
  article_text = parse_link(pagecontent, topic)
  i += 10
  with open('newtech.txt', 'a', encoding = 'utf-8') as f:
      f.write(article_text)

# 2. Предварительная обработка текстов
* прочитать каждый .txt файл в переменную,
* удалить из каждого текста последнее предложение про конференцию, 
* удалить короткие тексты с платным доступом
* очистить текст
* применить стоп-слова
* разбить на токены
* сделать морфологический разбор токенов для определения омонимов
* сделать лемматизацию для сравнения со словарем


In [ ]:
! pip install rusenttokenize
!pip install pymorphy2[fast]

In [ ]:
import re
import os
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download("stopwords", quiet=True)
from collections import defaultdict as dd
from operator import itemgetter
from nltk.corpus import stopwords
from rusenttokenize import ru_sent_tokenize
import string
import pymorphy2
from google.colab import files

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# предобработка - получаем морфологический разбор токенов из текста
morph_analyzer = pymorphy2.MorphAnalyzer()
russian_stopwords = stopwords.words('russian')
punctuation = string.punctuation + '«»'

def preprocess_tokenize(text):
    
    text_preprocessed_tokenized = []
        
    for sentence in ru_sent_tokenize(text):
        clean_words = [word.strip(punctuation) for word in word_tokenize(sentence)]
        clean_words = [word for word in clean_words if word]
        clean_words = [word.lower() for word in clean_words if word]

#        clean_lemmas = [morph_analyzer.parse(word)[0].normal_form for word in clean_words]
#        text_preprocessed_tokenized.extend(clean_lemmas)

        words_analized = [morph_analyzer.parse(token) for token in clean_words]
        text_preprocessed_tokenized.extend(words_analized)
        
    return text_preprocessed_tokenized






In [ ]:
# старая ячейка, которая читает и лемматизирует все файлы в текущей директории
'''articles = dict()
clean_lemmas = dict()
for filename in os.listdir():    # берем все txt файлы в папке
  if filename.endswith(".txt") : 
    with open(filename, 'r') as f:    
      file_text = f.read()
    #удаляем ненужный текст, который повторяется в каждой статье 
    file_text = file_text.replace('2-я Международная выставка и симпозиум V Международная Конференция Физико-химическая геотехнология - инновации и тенденции развития', '')
    file_text = file_text.replace('ISSN 1609-9192 (print)ISSN 2587-9138 (online)основан в 1992 году', '')
    #articles[filename] = ''
    #записываем в словарь только полные статьи (исключаем те, для которых закрыт бесплатный просмотр) 
    for article in file_text.split('\n\n') :
      if article != '' and 'У вас нет прав на просмотр этого материала.' not in article :
        clean_lemmas[filename.split('.')[0]] = preprocess_tokenize(article)
        #print(filename + ':' + str(len(clean_lemmas)))
        #articles[filename] = articles[filename] + file_text'''

In [ ]:
# выбираем файл
uploaded = files.upload()
for fn in uploaded.keys():
  filename = fn

morph_tokens = dict()

# читаем загруженный файл
with open(filename, 'r') as f:    
      file_text = f.read()
#удаляем ненужный текст, который повторяется в каждой статье 
file_text = file_text.replace('2-я Международная выставка и симпозиум V Международная Конференция Физико-химическая геотехнология - инновации и тенденции развития', '')
file_text = file_text.replace('ISSN 1609-9192 (print)ISSN 2587-9138 (online)основан в 1992 году', '')
file_text = file_text.replace('\xad', '')

i = 0
clean_articles_text = ''
#берем только полные статьи (исключаем те, для которых закрыт бесплатный просмотр) и вызываем функцию предобработки/токенизации 
for article in file_text.split('\n\n') :
  if article != '' and 'У вас нет прав на просмотр этого материала.' not in article :
  #  morph_tokens[i] = preprocess_tokenize(article)
  #  i = i + 1

    clean_articles_text = clean_articles_text + article 


morph_tokens = preprocess_tokenize(clean_articles_text) # морфологический анализ токенов текста

clean_lemmas = [analysis[0].normal_form for analysis in morph_tokens if analysis[0].normal_form not in russian_stopwords] # список чистых лемм текста



Saving OP.txt to OP (2).txt


In [ ]:
# смотрим, что получилось
# количество лемм
len(clean_lemmas)

12242

In [ ]:
#количество уникальных лемм
len(set(clean_lemmas))

3126

In [ ]:
clean_lemmas

In [ ]:
# сортировка лемм по частоте встречаемости 
res = sorted(set(clean_lemmas), key = lambda ele: clean_lemmas.count(ele))

In [ ]:
res[::-1]


# 3. Подготовка словарей
1. Словарь терминов
2. Частотный словарь русского языка


* **доработать словарь вручную**
* загрузить словарь из csv
* прочитать в список
* предобработка: нижний регистр, удаление повторов
* сравнить список лемм из текста и список из словаря и распечатать одинаковые

Если в словаре одно слово - сравнить с леммой.
Если в словаре словосочетание?
  - сравнить только существительное?
  - сравнить с биграммами из текста?


In [ ]:
import csv

# загружаем словарь
uploaded_dict = files.upload()
for fn in uploaded_dict.keys():
  filename_dict = fn

russian_terms = []

with open(filename_dict, newline=None) as csvfile:
  reader = csv.reader(csvfile, delimiter=';')
  for row in reader:
    russian_terms.append(row[0].lower())



Saving mining_terms_CL.csv to mining_terms_CL.csv


# 4. Получаем список слов из выбранного документа, которые есть в выбранном словаре 

In [ ]:
russian_terms = set(russian_terms)
result = set(clean_lemmas).intersection(russian_terms)
result
# добавить список английских соответствий из словаря в помощь переводчику


# 5. TF-IDF

In [ ]:
from operator import itemgetter
from nltk import word_tokenize
from nltk.corpus import stopwords
from rusenttokenize import ru_sent_tokenize
import string
import logging
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#составляем корпус из всех текстовых файлов в текущей директории
corpus = []
for filename in os.listdir():    # берем все txt файлы в папке
  if filename.endswith(".txt") : 
    with open(filename, 'r') as f:    
      corpus_file = f.read()
    #удаляем ненужный текст, который повторяется в каждой статье 
    #corpus_file = corpus_file.replace('2-я Международная выставка и симпозиум V Международная Конференция Физико-химическая геотехнология - инновации и тенденции развития', '')
    #corpus_file = corpus_file.replace('ISSN 1609-9192 (print)ISSN 2587-9138 (online)основан в 1992 году', '')
    #articles[filename] = ''
    #записываем в корпус только полные статьи (исключаем те, для которых закрыт бесплатный просмотр) 
    #for article in corpus_file.split('\n\n') :
    #  if article != '' and 'У вас нет прав на просмотр этого материала.' not in article :
    #    corpus.append(article)
    corpus.append(corpus_file)


In [ ]:
def preprocess_tokenize_lemmas(text):
    
    text_preprocessed_tokenized = []
        
    for sentence in ru_sent_tokenize(text):
        
        clean_words = [word.strip(string.punctuation) for word in word_tokenize(text)]
        clean_words = [word for word in clean_words if word]
        clean_words = [word.lower() for word in clean_words if word]
        clean_words = [word for word in clean_words if word not in russian_stopwords]
        
        clean_lemmas = [morph_analyzer.parse(word)[0].normal_form for word in clean_words]
        
        text_preprocessed_tokenized.extend(clean_lemmas)

    return text_preprocessed_tokenized

tfidf_vectorizer = TfidfVectorizer(tokenizer=preprocess_tokenize_lemmas)
tfidf_vectorizer.fit_transform(corpus)
feature_names = tfidf_vectorizer.get_feature_names()


#for document in corpus:
X = tfidf_vectorizer.transform([clean_articles_text])

tfidf_scores = [(feature_names[col], X[0, col]) for col in X.nonzero()[1]]
freq_list = [(word, freq) for word, freq in sorted(tfidf_scores, 
                                                    key=itemgetter(1), 
                                                    reverse=True)]

In [ ]:
freq_list

[('работа', 0.279721044782676),
 ('«', 0.22636385126921757),
 ('»', 0.2247469666172946),
 ('карьер', 0.19362910429384061),
 ('система', 0.18109108101537405),
 ('–', 0.17785731171152808),
 ('горный', 0.16492223449614424),
 ('метр', 0.16330534984422124),
 ('комбайн', 0.1332501362882344),
 ('вскрышной', 0.1332501362882344),
 ('порода', 0.13096765680576158),
 ('экскаватор', 0.12908606952922708),
 ('дорога', 0.12319306603623209),
 ('автосамосвал', 0.11867590263170875),
 ('разработка', 0.11641569493845474),
 ('карьерный', 0.11498019496714995),
 ('месторождение', 0.11479881028653177),
 ('оборудование', 0.10994815633076281),
 ('который', 0.10994815633076281),
 ('год', 0.10671438702691685),
 ('перемещение', 0.10618370235468678),
 ('транспортный', 0.09577353545716848),
 ('время', 0.09216242515961001),
 ('самосвал', 0.09054554050768703),
 ('отвал', 0.0876039580702095),
 ('полезный', 0.08407800189999509),
 ('условие', 0.07599357864038019),
 ('вскрыша', 0.07495320166213185),
 ('датчик', 0.073915839